In [ ]:
!pip install transformers wget

     |████████████████████████████████| 2.1MB 6.8MB/s 
     |████████████████████████████████| 901kB 30.7MB/s 
     |████████████████████████████████| 3.3MB 47.2MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=b04289b7dbe07e900875bede7bd7a6052f0026b0ff42a2831781c68e4a0d0162
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import os
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel
import wget
import copy


# MAX_LEN = 300
MAX_LEN = 384
EPOCHS = 1
VERBOSE = 1
BATCH_SIZE = 20

In [ ]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", lowercase=False)
save_path = "bert-base-multilingual-cased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert-base-multilingual-cased/vocab.txt", lowercase=False)

In [ ]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json', out='./bert-base-multilingual-cased/')

'./bert-base-multilingual-cased//bert-base-multilingual-cased-config.json'

In [ ]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-config.json', './bert-base-multilingual-cased/config.json')

In [ ]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-tf_model.h5', out='./bert-base-multilingual-cased/')

'./bert-base-multilingual-cased//bert-base-multilingual-cased-tf_model.h5'

In [ ]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5', './bert-base-multilingual-cased/tf_model.h5')

In [ ]:
class TFBERTQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBERTQuestionAnswering, self).__init__()
        
        self.encoder = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.start_logit = tf.keras.layers.Dense(num_class, name="start_logit", use_bias=False)
        self.end_logit = tf.keras.layers.Dense(num_class, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
        start_logits = self.start_logit(embedding)
        start_logits = self.flatten(start_logits)
        
        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)
    
        return start_probs, end_probs

In [ ]:
korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

Some layers from the model checkpoint at ./bert-base-multilingual-cased/ were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ./bert-base-multilingual-cased/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
korquad_model.compile(optimizer=optimizer, loss=[loss, loss])

In [ ]:
build_input = [np.array([np.array([0,0])]),np.array([np.array([0,0])]),np.array([np.array([0,0])])]

In [ ]:
build_output = [np.array([[0]]), np.array([[0]])]

In [ ]:
korquad_model.fit(
    build_input,
    build_output,
    epochs=1,  # For demonstration, 3 epochs are recommended
    verbose=1,
    batch_size=16,
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
korquad_model.summary()

Model: "tfbert_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model (TFBertModel)  multiple                  177853440 
_________________________________________________________________
start_logit (Dense)          multiple                  768       
_________________________________________________________________
end_logit (Dense)            multiple                  768       
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
activation (Activation)      multiple                  0         
Total params: 177,854,976
Trainable params: 177,854,976
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# korquad_model.load_weights('./bert_korquad3.h5')
korquad_model.load_weights('/content/drive/MyDrive/bert_weights/bert_korquad3.h5')

In [ ]:
def convert_data(question, context):

    context = " ".join(str(context).split())
    question = " ".join(str(question).split())    
    
    tokenized_context = tokenizer.encode(context)
    tokenized_question = tokenizer.encode(question)

    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []


    # Create inputs
    input_ids = tokenized_context.ids + tokenized_question.ids[1:]
    token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
    tokenized_question.ids[1:]
    )
    attention_mask = [1] * len(input_ids)

    # Pad and create attention masks.
    # Skip if truncation is needed
    padding_length = MAX_LEN - len(input_ids)
    if padding_length > 0:  # pad
        input_ids = input_ids + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
    elif padding_length < 0:
        pass

    input_ids_list.append(input_ids)
    token_type_ids_list.append(token_type_ids)
    attention_mask_list.append(attention_mask)

    input_ids_list = np.array(input_ids_list)
    token_type_ids_list = np.array(token_type_ids_list)
    attention_mask_list = np.array(attention_mask_list)

   
    return [input_ids_list, token_type_ids_list, attention_mask_list]

In [ ]:
def predict_answer(question, context):
    
    context_origin = copy.deepcopy(context)

    input = convert_data(question, context)
    answer_start, answer_end = korquad_model.predict(input)

    answer_start_idx =  np.argmax(answer_start)
    answer_end_idx = np.argmax(answer_end)

    context = " ".join(str(context).split())
    context_tokens = tokenizer.encode(context).tokens
    answer_tokens = []
    for i in range(answer_start_idx, answer_end_idx+1):
        answer_tokens.append(tokenizer.encode(context).tokens[i])
    
    answer_sentence = ''

    for answer_token in answer_tokens:
        if answer_token.startswith("##"):
            answer_token = answer_token.replace("##","")
        else:
            answer_token = " " + answer_token
          
        answer_sentence += answer_token

    print(f'Context :\n{context_origin}\n')
    print(f'Question : {question}\n')
    print(f'Answer :{answer_sentence}')    

In [ ]:
# 출처 : 나무위키
context = '''
마케도니아 왕국 출신의 그리스 철학자.[3] 1998년 저명한 현대 철학자들이 벌인 "서구 철학계에서 가장 큰 영향을 끼친 철학자"를 뽑는 투표에서 1위 차지.[4][5]

스승인 플라톤과 함께 2천여 년 서양철학사에서 가장 중요한 역할을 차지하는 위인이다.[6] 형이상학, 논리학, 정치철학, 윤리학, 자연철학, 과학, 생물학 등 인간이 할 수 있는 모든 분야에 통달하고 그것들의 기초를 마련한 철학사의 거인.

스탠퍼드 철학 백과에서는 아리스토텔레스를 "모든 시대의 가장 위대한 철학자 중 하나"로, 브리태니커 백과사전에서는 "서구 역사상 가장 위대한 지성인 중 하나"로 소개하고 있다.
'''
question = '아리스토텔레스의 스승은 누구인가?'
predict_answer(question, context)

Context :

마케도니아 왕국 출신의 그리스 철학자.[3] 1998년 저명한 현대 철학자들이 벌인 "서구 철학계에서 가장 큰 영향을 끼친 철학자"를 뽑는 투표에서 1위 차지.[4][5]

스승인 플라톤과 함께 2천여 년 서양철학사에서 가장 중요한 역할을 차지하는 위인이다.[6] 형이상학, 논리학, 정치철학, 윤리학, 자연철학, 과학, 생물학 등 인간이 할 수 있는 모든 분야에 통달하고 그것들의 기초를 마련한 철학사의 거인.

스탠퍼드 철학 백과에서는 아리스토텔레스를 "모든 시대의 가장 위대한 철학자 중 하나"로, 브리태니커 백과사전에서는 "서구 역사상 가장 위대한 지성인 중 하나"로 소개하고 있다.


Question : 아리스토텔레스의 스승은 누구인가?

Answer : 플라톤


In [ ]:
context = '''
마케도니아 왕국 출신의 그리스 철학자.[3] 1998년 저명한 현대 철학자들이 벌인 "서구 철학계에서 가장 큰 영향을 끼친 철학자"를 뽑는 투표에서 1위 차지.[4][5]

스승인 플라톤과 함께 2천여 년 서양철학사에서 가장 중요한 역할을 차지하는 위인이다.[6] 형이상학, 논리학, 정치철학, 윤리학, 자연철학, 과학, 생물학 등 인간이 할 수 있는 모든 분야에 통달하고 그것들의 기초를 마련한 철학사의 거인.

스탠퍼드 철학 백과에서는 아리스토텔레스를 "모든 시대의 가장 위대한 철학자 중 하나"로, 브리태니커 백과사전에서는 "서구 역사상 가장 위대한 지성인 중 하나"로 소개하고 있다.
'''
question = '아리스토텔레스의 출신은 어디?'
predict_answer(question, context)

Context :

마케도니아 왕국 출신의 그리스 철학자.[3] 1998년 저명한 현대 철학자들이 벌인 "서구 철학계에서 가장 큰 영향을 끼친 철학자"를 뽑는 투표에서 1위 차지.[4][5]

스승인 플라톤과 함께 2천여 년 서양철학사에서 가장 중요한 역할을 차지하는 위인이다.[6] 형이상학, 논리학, 정치철학, 윤리학, 자연철학, 과학, 생물학 등 인간이 할 수 있는 모든 분야에 통달하고 그것들의 기초를 마련한 철학사의 거인.

스탠퍼드 철학 백과에서는 아리스토텔레스를 "모든 시대의 가장 위대한 철학자 중 하나"로, 브리태니커 백과사전에서는 "서구 역사상 가장 위대한 지성인 중 하나"로 소개하고 있다.


Question : 아리스토텔레스의 출신은 어디?

Answer : 마케도니아 왕국


In [ ]:
context = '''
마케도니아 왕국 출신의 그리스 철학자.[3] 1998년 저명한 현대 철학자들이 벌인 "서구 철학계에서 가장 큰 영향을 끼친 철학자"를 뽑는 투표에서 1위 차지.[4][5]

스승인 플라톤과 함께 2천여 년 서양철학사에서 가장 중요한 역할을 차지하는 위인이다.[6] 형이상학, 논리학, 정치철학, 윤리학, 자연철학, 과학, 생물학 등 인간이 할 수 있는 모든 분야에 통달하고 그것들의 기초를 마련한 철학사의 거인.

스탠퍼드 철학 백과에서는 아리스토텔레스를 "모든 시대의 가장 위대한 철학자 중 하나"로, 브리태니커 백과사전에서는 "서구 역사상 가장 위대한 지성인 중 하나"로 소개하고 있다.
'''
question = '브리태니커 백과사전에서는 아리스토텔레스를 어떻게 소개하고 있어?'
predict_answer(question, context)

Context :

마케도니아 왕국 출신의 그리스 철학자.[3] 1998년 저명한 현대 철학자들이 벌인 "서구 철학계에서 가장 큰 영향을 끼친 철학자"를 뽑는 투표에서 1위 차지.[4][5]

스승인 플라톤과 함께 2천여 년 서양철학사에서 가장 중요한 역할을 차지하는 위인이다.[6] 형이상학, 논리학, 정치철학, 윤리학, 자연철학, 과학, 생물학 등 인간이 할 수 있는 모든 분야에 통달하고 그것들의 기초를 마련한 철학사의 거인.

스탠퍼드 철학 백과에서는 아리스토텔레스를 "모든 시대의 가장 위대한 철학자 중 하나"로, 브리태니커 백과사전에서는 "서구 역사상 가장 위대한 지성인 중 하나"로 소개하고 있다.


Question : 브리태니커 백과사전에서는 아리스토텔레스를 어떻게 소개하고 있어?

Answer : 서구 역사상 가장 위대한 지성인 중 하나


In [ ]:
context = '''
철수와 영희는 유치원에서 처음 만난 후 죽마고우가 됐다.
어느덧 세월이 흘러 철수와 영희는 25살이 되었다.
하지만 그 둘은 여전히 사이가 좋다.
'''
question = '철수와 영희는 어디에서 처음 만났어?'
predict_answer(question, context)

Context :

철수와 영희는 유치원에서 처음 만난 후 죽마고우가 됐다.
어느덧 세월이 흘러 철수와 영희는 25살이 되었다.
하지만 그 둘은 여전히 사이가 좋다.


Question : 철수와 영희는 어디에서 처음 만났어?

Answer : 유치원에
